# 엔드투엔드 NLP: 뉴스 헤드라인 분류기(SageMaker 버전)

_**네 가지 도메인 간 뉴스 헤드라인을 분류하기 위해 Keras 기반 모델 학습**_

이 노트북은 SageMaker 스튜디오의 `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)`  커널 또는 클래식 세이지메이커 노트북 인스턴스의 `conda_tensorflow2_p37`에서 잘 작동합니다.


---

이전 '로컬 버전' 노트북에 이어, 여기서는 리소스를 더 잘 활용하기 위해 별도의 인프라에서 모델 훈련 및 배포를 트리거하는 방법을 보여줍니다.

pip 버전에 대한 경고는 무시해도 됩니다.

In [ ]:
# First install some libraries which might not be available across all kernels (e.g. in Studio):
!pip install "ipywidgets<8"

### News Aggregator 데이터셋 다운로드

[AWS의 오픈 데이터 레지스트리](https://registry.opendata.aws/fast-ai-nlp/) 퍼블릭 리포지토리에서 **FastAI AG 뉴스** 데이터 세트를 다운로드합니다. 이 데이터 세트에는 뉴스 헤드라인과 그에 해당하는 클래스의 표가 포함되어 있습니다.


In [ ]:
%%time
local_dir = "data"
# Download the AG News data from the Registry of Open Data on AWS.
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-nlp/ag_news_csv.tgz {local_dir} --no-sign-request

# Un-tar the AG News data.
!tar zxf {local_dir}/ag_news_csv.tgz -C {local_dir}/ --strip-components=1 --no-same-owner
print("Done!")

### 데이터 세트를 시각화해 봅시다.

데이터 처리 작업을 위해 ag_news_csv/train.csv 파일을 Pandas 데이터 프레임에 로드하겠습니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re

import numpy as np
import pandas as pd
import util.preprocessing

In [ ]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv(f"{local_dir}/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)
# make the category classes more readable
mapping = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tech"}
df = df.replace({"CATEGORY": mapping})
df.head()

이 연습에서는 **아래 내용만 사용하겠습니다**:

- 뉴스 기사의 **제목**(헤드라인)을 입력으로 사용합니다.
- 대상 변수로 **카테고리**를 사용합니다.


In [ ]:
df["CATEGORY"].value_counts()

The dataset has **four article categories** with equal weighting:

- Business
- Sci/Tech
- Sports
- World


## 자연어 전처리

텍스트 데이터를 알고리즘이 모델을 생성하는 데 사용할 수 있는 숫자 형식으로 변환하기 위해 몇 가지 기본 처리를 수행합니다.

라벨 더미 인코딩, 문서 토큰화, 입력 특징 차원에 대한 고정 시퀀스 길이 설정, 고정 길이 입력 벡터를 갖도록 문서 패딩과 같은 NLP 워크로드에 대한 일반적인 사전 처리를 수행합니다.

### Dummy Encode the Labels


In [ ]:
encoded_y, labels = util.preprocessing.dummy_encode_labels(df, "CATEGORY")
print(labels)
print(encoded_y)

In [ ]:
df["CATEGORY"].iloc[0]

In [ ]:
encoded_y[0]

###  토큰화 및 고정 시퀀스 길이 설정하기

개별 문자가 아닌 보다 의미 있는 단어 수준에서 입력을 설명하고, 입력 특징 차원의 고정된 길이를 보장하고자 합니다.


In [ ]:
processed_docs, tokenizer = util.preprocessing.tokenize_and_pad_docs(df, "TITLE")

In [ ]:
df["TITLE"].iloc[0]

In [ ]:
processed_docs[0]

### 단어 임베딩 가져오기

단어를 숫자 형태로 표현하기 위해 어휘의 각 단어에 대해 미리 학습된 벡터 표현을 사용합니다: 이 경우 영어 이외의 다양한 언어에도 사용할 수 있는 [pre-trained word embeddings from FastText](https://fasttext.cc/docs/en/crawl-vectors.html)을 사용할 것입니다.

SageMaker에 내장된 [BlazingText algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html)을 사용하여 도메인별 맞춤 단어 임베딩을 학습할 수도 있습니다. 방법을 보여주는 예제 노트북은 공식 [blazingtext_word2vec_text8 sample](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/blazingtext_word2vec_text8)을 참조하세요.

> ⚠️ 이 셀을 로컬 노트북의 `get_word_embeddings()` 셀과 동시에 실행하면 간혹 파일 형식 오류가 발생할 수 있습니다. 이 경우, 이 셀을 다시 실행하기 전에 `data/embeddings` 폴더를 삭제해 보세요.

In [ ]:
%%time
embedding_matrix = util.preprocessing.get_word_embeddings(tokenizer, f"{local_dir}/embeddings")

In [ ]:
np.save(
    file=f"{local_dir}/embeddings/docs-embedding-matrix",
    arr=embedding_matrix,
    allow_pickle=False,
)
vocab_size = embedding_matrix.shape[0]
print(embedding_matrix.shape)

### 학습 및 테스트 세트 분할

마지막으로 데이터를 모델 훈련 세트와 평가 세트로 나눠야 합니다:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    processed_docs,
    encoded_y,
    test_size=0.2,
    random_state=42,
)

In [ ]:
os.makedirs(f"{local_dir}/train", exist_ok=True)
np.save(f"{local_dir}/train/train_X.npy", X_train)
np.save(f"{local_dir}/train/train_Y.npy", y_train)
os.makedirs(f"{local_dir}/test", exist_ok=True)
np.save(f"{local_dir}/test/test_X.npy", X_test)
np.save(f"{local_dir}/test/test_Y.npy", y_test)

## 실행 역할, 세션 및 S3 버킷 설정 ###

SageMaker 학습 작업의 기본 데이터 소스는 (거의) 항상 Amazon S3입니다. SageMaker 학습 작업에 사용할 수 있도록 사전 처리된 데이터를 여기에 업로드해야 합니다. 

지정하는 것부터 시작하겠습니다:
- 학습 및 모델 데이터에 사용할 S3 버킷과 접두사를 지정합니다. 이 버킷은 노트북 인스턴스, 학습 및 호스팅과 동일한 리전 내에 있어야 합니다. 버킷을 지정하지 않으면 SageMaker SDK는 동일한 리전에서 미리 정의된 명명 규칙에 따라 기본 버킷을 생성합니다. 
- SageMaker에 데이터 액세스 권한을 부여하는 데 사용되는 IAM 역할 ARN입니다. SageMaker 파이썬 SDK에서 **get_execution_role** 메서드를 사용하여 가져올 수 있습니다.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()
bucket_name = sess.default_bucket()

## Amazon S3에 데이터 업로드
S3에 데이터를 업로드하는 한 가지 방법은 대상 버킷과 소스 디렉터리의 콘텐츠를 동기화하는 상위 수준 [aws s3 sync](https://docs.aws.amazon.com/cli/latest/userguide/cli-services-s3-commands.html#using-s3-commands-managing-objects-sync) 명령을 사용하는 것입니다. 이 명령은 다음과 같은 옵션도 지원합니다:

- ```--delete```, 대상에서 소스에 없는 모든 오브젝트를 제거합니다.
- 소스 폴더에서 모든 것을 복사하는 대신 파일/개체를 필터링하려면 ```--exclude``` 및 ```--include```를 사용합니다.

Python을 통해서도 S3로 작업할 수 있지만(예: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html) 또는 [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html)), S3 동기화 CLI는 기본적으로 *멀티 스레드*를 지원합니다: 따라서 더 복잡한 코드를 도입하지 않고도 노트북에서 빠르게 전송할 수 있습니다.

In [ ]:
!aws s3 sync --quiet --delete {local_dir} s3://{bucket_name}/news --exclude "*" --include "*.npy"
print("Done!")

## 데이터 입력("채널") 구성
로컬 버전의 노트북([Headline Classifier Local.ipynb](Headline%20Classifier%20Local.ipynb))에는 학습, 테스트, 임베딩의 세 가지 데이터 입력이 있습니다. SageMaker 용어로 각 입력 데이터는 "채널"입니다.


In [ ]:
train_channel = f"s3://{bucket_name}/news/train"
test_channel = f"s3://{bucket_name}/news/test"
embeddings_channel = f"s3://{bucket_name}/news/embeddings"

inputs = {"train": train_channel, "test": test_channel, "embeddings": embeddings_channel}
print(inputs)

## SageMaker에서 차별화된 인프라로 학습하기

이번에는 이전 노트북의 모델 빌드 및 학습 코드([Headline Classifier Local.ipynb](Headline%20Classifier%20Local.ipynb))를 **src** 디렉터리의 [**main.py**](src/main.py) 스크립트에 패키징했습니다.

이 스크립트 파일에서 모델을 학습하고 배포하기 위해 상위 레벨의 [SageMaker Python SDK를 통한 텐서플로우 프레임워크 컨테이너](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html)를 사용할 것입니다.

## 사전 빌드된 컨테이너로 Amazon SageMaker에서 Tensorflow 스크립트를 실행하는 방법 ###

AWS는 주요 머신러닝 프레임워크에서 프로젝트를 빠르게 구축할 수 있도록 사전 패키지된 Docker 이미지 세트를 제공합니다: [SageMaker 프레임워크 컨테이너](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html).

이 컨테이너는 GPU 드라이버, 서빙 스택 구현, 핵심 라이브러리 등과 같은 기본 설정을 처리하므로 트레이닝 프로세스 및 추론 동작 오버라이드를 위한 Python 스크립트만 간단히 삽입하면 됩니다. 심지어 컨테이너 이미지에 빌드할 필요 없이 시작 시 동적으로 설치할 추가 종속성을 지정하는 *requirements.txt* 파일을 제공할 수도 있습니다.

**따라서 첫 번째 작업은 스크립트와 런타임 간의 인터페이스**를 이해하는 것입니다: 스크립트가 입력 데이터를 어떻게 읽을 것인가? 매개변수는? 결과를 어디에 저장해야 할까요?

#### 학습 중 컨테이너 실행

Amazon SageMaker가 트레이닝을 실행할 때, 트레이닝 스크립트(entry_point 입력)는 일반 Python 프로그램과 마찬가지로 실행됩니다. 여러 파일이 `/opt/ml` 디렉토리에 배치되어 있습니다. 이 파일들은 스크립트 내에서 액세스할 수 있는 위치입니다. 이 파일의 사용 예는 [**main.py**](src/main.py)에서 확인할 수 있습니다:


    /opt/ml
    |-- code
    |   `-- <our script(s)>
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure
 
##### 입력

* `/opt/ml/input/config`에는 프로그램 실행 방식을 제어하는 정보가 들어 있습니다. `hyperparameters.json`은 하이퍼파라미터 이름을 값으로 변환한 JSON 형식의 딕셔너리입니다. 이 값은 항상 문자열이므로 변환해야 할 수도 있습니다. `resourceConfig.json` 은 분산 학습에 사용되는 네트워크 레이아웃을 설명하는 JSON 형식의 파일입니다. scikit-learn은 분산 학습을 지원하지 않으므로 여기서는 무시하겠습니다.
* (파일 모드의 경우) `/opt/ml/input/data/<channel_name>/`에는 해당 채널의 입력 데이터가 포함됩니다. 채널은 CreateTrainingJob 호출에 따라 생성되지만 일반적으로 채널이 알고리즘이 예상하는 것과 일치하는 것이 중요합니다. 각 채널의 파일은 S3 키 구조로 표시된 트리 구조를 유지하면서 S3에서 이 디렉토리로 복사됩니다. 
* (파이프 모드의 경우) `/opt/ml/input/data/<channel_name>_<epoch_number>`는 주어진 에포크에 대한 파이프입니다. 에포크는 0에서 시작하여 읽을 때마다 하나씩 올라갑니다. 실행할 수 있는 에포크 수에는 제한이 없지만 다음 에포크를 읽기 전에 각 파이프를 닫아야 합니다.

##### 출력
    
* `/opt/ml/model/`은 알고리즘이 생성하는 모델을 작성하는 디렉토리입니다. 모델은 원하는 모든 형식이 가능합니다. 단일 파일 또는 전체 디렉토리 트리가 될 수 있습니다. 세이지메이커는 이 디렉토리에 있는 모든 파일을 압축된 타르 아카이브 파일로 패키징합니다. 이 파일은 `DescribeTrainingJob` 결과에서 반환된 S3 위치에서 사용할 수 있습니다.
* `/opt/ml/output`은 알고리즘이 작업이 실패한 이유를 설명하는 `실패` 파일을 작성할 수 있는 디렉터리입니다. 이 파일의 내용은 `DescribeTrainingJob` 결과의 `FailureReason` 필드에 반환됩니다. 성공한 작업의 경우 이 파일은 무시되므로 이 파일을 작성할 이유가 없습니다.

#### 추가 정보

자세한 내용은 다음을 참조하세요:

- [SageMaker Python SDK guide for TensorFlow](https://sagemaker.readthedocs.io/en/stable/using_tf.html), 텐서플로우 프레임워크 클래스에 대한 [API 문서](https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html)를 참고하세요.
- 기본 컨테이너 이미지를 정의하는 GitHub의 [AWS Deep Learning Containers repository](https://github.com/aws/deep-learning-containers).
- 학습 및 서비스를 위한 프레임워크 코드에 대한 자세한 내용은 오픈 소스 SageMaker [TensorFlow Training Toolkit](https://github.com/aws/sagemaker-tensorflow-training-toolkit) 및 [TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container)를 참조하세요.

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow as TensorFlowEstimator

스크립트는 별도의 컨테이너에서 실행되지만, 필요한 매개변수는 SageMaker를 통해 전달할 수 있습니다:


In [ ]:
hyperparameters = {"epochs": 5, "max_seq_len": 40, "num_classes": 4}

이제 `TensorFlow` 추정기(estimator) 객체가 있고, 이 객체에 대한 하이퍼 파라미터를 설정했으며, 알고리즘과 연결된 데이터 채널이 있습니다. 이제 남은 일은 알고리즘을 학습하는 것입니다. 다음 명령은 몇 가지 단계로 구성된 이 학습을 실행합니다:

- 먼저, `TensorFlow` 추정기(estimator) 클래스를 생성할 때 요청했던 인스턴스를 프로비저닝하고 적절한 라이브러리를 사용하여 설정합니다.
- 그런 다음 채널의 데이터를 인스턴스로 다운로드합니다.
- 이 작업이 완료되면 학습 작업이 코드 실행을 시작합니다.

프로비저닝과 데이터 다운로드는 데이터의 크기와 컨테이너 이미지에 따라 다소 시간이 걸립니다. 

작업이 완료되면 "작업 완료" 메시지가 나타납니다. 학습된 모델은 추정기(estimator)에서 'output_path'로 설정된 S3 버킷에서 찾을 수 있습니다.

여기서는 컴퓨팅 최적화(CPU) `ml.c5.xlarge` 인스턴스 유형을 사용하여 학습 작업을 실행합니다. 더 큰 딥 러닝 모델이나 데이터 세트의 경우, `ml.p*` 또는 `ml.g*`와 같은 GPU 가속 유형을 사용하여 학습 속도를 더욱 높일 수 있습니다.


In [ ]:
%%time

metric_definitions = [
    {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
    {"Name": "accuracy", "Regex": "acc: ([0-9\\.]+)"},
    {"Name": "validation:loss", "Regex": "Validation.*loss=([0-9\\.]+)"},
    {"Name": "validation:accuracy", "Regex": "Validation.*acc=([0-9\\.]+)"},
]

estimator = TensorFlowEstimator(
    framework_version="2.4",
    py_version="py37",
    instance_count=1,
    instance_type="ml.c5.xlarge",
    role=role,
    entry_point="main.py",
    source_dir="./src",
    distribution={"parameter_server": {"enabled": True}},
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    base_job_name="news-keras",
    max_run=20 * 60,  # Maximum allowed active runtime
    ## Here Spot is left OFF by default to avoid delays, but easy to turn on by un-commenting:
    # use_spot_instances=True,  # Use spot instances to reduce cost
    # max_wait=30*60,  # Maximum clock time (including spot wait time)
)

estimator.fit(inputs)

While the training job is running take a minute to look at the `main.py` script. You can see how we have adapted the our original local code from [Headline Classifier Local.ipynb](Headline%20Classifier%20Local.ipynb) to run on Sagemaker.

학습 작업이 실행되는 동안 잠시 시간을 내어 `main.py` 스크립트를 살펴보세요. [Headline Classifier Local.ipynb](Headline%20Classifier%20Local.ipynb)의 원래 로컬 코드를 Sagemaker에서 실행하도록 어떻게 조정했는지 확인할 수 있습니다.

## 모델 사용: 호스팅 / 추론
학습이 완료되면 학습된 모델을 Amazon SageMaker에서 호스팅하는 [real-time inference endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)로 배포할 수 있습니다. 이렇게 하면 필요에 따라 모델에서 예측(또는 추론)을 쿼리할 수 있습니다.

참고하세요:

- 배치 추론 상황(예: 테스트 데이터 세트에서 메트릭 계산)의 경우, 대신 [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)을 사용하는 것이 좋습니다. 자세한 내용은 SageMaker 파이썬 SDK 문서에서 [세이지메이커 배치 변환 사용하기](https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform)를 참고하세요.
- 학습한 것과 동일한 **유형**의 인스턴스를 사용할 필요는 없습니다. 이 테스트 엔드포인트는 매우 적은 트래픽을 처리할 것이므로 더 저렴한 유형을 사용할 수 있습니다:


In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
)

### 이제 모델이 프로덕션 환경에서 RESTful API로 작동합니다!

몇 가지 예제 헤드라인으로 모델을 평가해 봅시다...

위젯을 사용하는 데 어려움이 있다면 파이썬에서 `classify()` 함수를 호출하면 됩니다.

헤드라인을 창의적으로 만들 수 있습니다!

In [ ]:
import ipywidgets as widgets
from IPython import display
from tensorflow.keras.preprocessing.sequence import pad_sequences


def classify(text):
    """Classify a headline and print the results"""
    encoded_example = tokenizer.texts_to_sequences([text])
    # Pad documents to a max length of 40 words
    max_length = 40
    padded_example = pad_sequences(encoded_example, maxlen=max_length, padding="post")
    result = predictor.predict(padded_example.tolist())
    print(result)
    ix = np.argmax(result["predictions"])
    print(f"Predicted class: '{labels[ix]}' with confidence {result['predictions'][0][ix]:.2%}")


# Either try out the interactive widget:
interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    ),
)
interaction.widget.children[1].description = "Classify!"

In [ ]:
# Or just use the function to classify your own headline:
classify("Retailers are expanding after the recent economic growth")

## 정리

학습이 완료되는 즉시 리소스를 삭제하는 학습 작업과 달리, 실시간 엔드포인트 배포는 엔드포인트를 종료할 때까지 인스턴스를 프로비저닝합니다....

따라서 리소스를 아껴서 사용하고 더 이상 필요하지 않으면 리소스를 삭제해야 합니다. 아래 셀의 댓글을 취소하고 실행하여 엔드포인트를 삭제할 수 있습니다:

In [ ]:
# predictor.delete_endpoint(delete_endpoint_config=True)

## (옵션) 자동 하이퍼파라미터 최적화 - HPO

모델 성능을 조정하기 위해 매개변수를 수동으로 조정하는 대신 SageMaker의 도움을 받을 수 있습니다.

SageMaker에 아래 정보를 간단히 알려주면 됩니다:

- 각 매개변수의 유형과 허용 범위,
- 최적화하고자 하는 메트릭, 그리고
- 전략 및 리소스 제약 조건

...그러면 서비스가 최적의 조합을 찾을 수 있도록 작업을 설정합니다.

### (하이퍼-)파라미터 정의


In [ ]:
from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
    IntegerParameter,
)

hyperparameter_ranges = {
    "epochs": IntegerParameter(2, 7),
    "learning_rate": ContinuousParameter(0.01, 0.2),
}

### 목표 메트릭

SageMaker의 '메트릭'은 정규 표현식을 통해 작업의 콘솔 출력에서 스크랩됩니다.

모니터링할 메트릭을 여러 개 정의할 수 있지만, HPO에서는 그 중 정확히 한 개를 최적화할 **목표** 메트릭으로 지정해야 합니다:


In [ ]:
# (See metric_definitions above)
objective_metric_name = "validation:accuracy"
objective_type = "Maximize"

### 튜닝 작업 시작

위에서 이미 Estimator를 정의했으므로 약간의 조정을 통해 구성된 것을 재사용하겠습니다.

추정기(Estimator)의 '하이퍼파라미터'가 기본값으로 사용되며, 적절한 경우 하이퍼파라미터튜너로 재정의된다는 점에 유의하세요.

In [ ]:
# Keep per-job resources modest, so that parallel jobs don't hit any limits:
estimator.instance_type = "ml.c5.xlarge"
estimator.instance_count = 1

In [ ]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    base_tuning_job_name="news-hpo-keras",
    max_jobs=6,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

tuner.fit(inputs)

### 진행 상황 확인

HPO 작업은 완료하는 데 시간이 오래 걸릴 수 있으며, 여러 인스턴스에서 각각 여러 훈련 작업을 병렬로 실행할 수 있습니다... 그렇기 때문에 위의 `fit()` 호출은 잠재적으로 혼란스러울 수 있는 통합 로그 스트림을 표시하지 않으며, `wait=False` 매개 변수를 추가하면 완료될 때까지 기다리지 않을 수 있습니다.

[**SageMaker 콘솔**](https://console.aws.amazon.com/sagemaker/home#/hyper-tuning-jobs)의 Training > Hyperparameter Tuning Job 페이지로 이동하여 목록에서 작업을 선택합니다.

HPO 실행을 위해 트리거된 모든 트레이닝 작업과 전체 요약 지표를 확인할 수 있습니다.

물론 API/SDK를 통해서도 이 정보에 액세스할 수 있습니다. 예를 들어 아래와 같이 HPO가 완료될 때까지 기다릴 수 있습니다:


In [ ]:
import time

import boto3

# Wait until HPO is finished
hpo_state = None
smclient = boto3.Session().client("sagemaker")

while hpo_state is None or hpo_state == "InProgress":
    if hpo_state is not None:
        print("-", end="")
        time.sleep(60)  # Poll once every 1 min
    hpo_state = smclient.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
    )["HyperParameterTuningJobStatus"]

print("\nHPO state:", hpo_state)

### 모델 사용하기

`estimator`와 마찬가지로 `tuner.deploy()`를 호출하여 HPO 실행에서 찾은 가장 성능이 좋은 모델에서 엔드포인트와 `predictor`를 생성할 수 있습니다.


## 검토

이 노트북에서는 로컬 코드를 리팩토링하여 SageMaker를 사용하여 동일한 Keras 모델을 학습하고 배포했습니다.

이 접근 방식의 몇 가지 이점은 다음과 같습니다:

- 학습 작업 기간 동안만 **전문 컴퓨팅 리소스(예: 고성능 또는 GPU 가속 인스턴스)를 자동으로 프로비저닝할 수 있습니다: 활용도가 낮은 리소스를 방치하지 않고 학습에서 우수한 성능을 얻을 수 있습니다.
- 사용자가 무엇이 효과가 있고 무엇이 효과가 없었는지를 기록해야 하는 로컬 노트북 실험과 달리, 학습 작업의 이력(매개변수, 메트릭, 출력 등)이 자동으로 추적됩니다.
- 학습된 모델은 단 한 번의 SDK 호출로 프로덕션에 바로 사용할 수 있는 안전한 웹 엔드포인트에 배포할 수 있습니다: 동작을 사용자 정의하려는 경우가 아니라면 컨테이너나 웹 애플리케이션 패키징이 필요하지 않습니다.
